In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import scipy
import sys
sys.path.append("..")
import os
pd.set_option("max_columns", 10000)

%pylab inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import json
from IPython import display

from tqdm import tqdm, tqdm_notebook, tqdm_pandas
tqdm.pandas()

import copy

from Scripts.saving import submit

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from Scripts.utils import *

import json

import featuretools as ft

Populating the interactive namespace from numpy and matplotlib


In [2]:
base_dir = ".."
input_dir = open( os.path.join(base_dir, "datadir.txt"), "r").read()[:-1]
pred_dir = os.path.join(base_dir, "predictions")
processed_dir = os.path.join(input_dir, "processed")
!tree {input_dir}

/home/furfa/work/ai-academy2019/input
├── ai-academy-2019-skill-prediction-data-csv-v1.zip
├── ai-academy-2019-skill-prediction-data-jsonlines.zip
├── dota2_abilities.csv
├── dota2_heroes.csv
├── dota2_items.csv
├── dota2_skill_test.csv
├── dota2_skill_test.jsonlines
├── dota2_skill_train.csv
├── dota2_skill_train.jsonlines
└── processed
    ├── heroes_proc.csv
    ├── items_proc.csv
    ├── test_all_JSON.csv
    ├── test_levelup.csv
    ├── test_proc.csv
    ├── test_team.csv
    ├── test_timeseries.csv
    ├── train_all_JSON.csv
    ├── train_levelup.csv
    ├── train_proc.csv
    ├── train_team.csv
    └── train_timeseries.csv

1 directory, 21 files


In [3]:
# aditional
abilities = pd.read_csv( os.path.join(input_dir, 'dota2_abilities.csv') ).drop(["dname", "desc"], axis=1)
items = pd.read_csv( os.path.join(input_dir,"processed", 'items_proc.csv') )
heroes = pd.read_csv( 
    os.path.join(input_dir,"processed", 'heroes_proc.csv') 
).set_index("hero_id").drop('Unnamed: 0', axis=1)

In [4]:
y_train = pd.read_csv( os.path.join(input_dir,"processed", 'train_all_JSON.csv'), index_col=0 ).skilled

In [5]:
def pipe(type_ = "train"):
    data = pd.read_csv( os.path.join(input_dir,"processed", f"{type_}_all_JSON.csv"), index_col=0 )
    
    if type_ == "train":
        data.drop(["skilled"], 1, inplace = True)
    
    json_columns = ['damage_targets', 
                'final_items', 
                'ability_upgrades',
                'series',
                'dire_heroes',
                'level_up_times',
                'item_purchase_log',
                'radiant_heroes'
               ]
    
    def tr(x):
        if x=='dire':
            return 0
        elif x=='radiant':
            return 1
    data["player_team"] = data["player_team"].apply(tr)
    data["winner_team"] = data["winner_team"].apply(tr)
    data_json = data[json_columns]
    data.drop(json_columns, axis=1, inplace=True)
    
    def prep_heroes(ser, team):
        new_data = {f"{team}{i}":list() for i in range(1,6)}
        for line in ser:
            line = json.loads(line)
            line.sort()
            for i in range(1,6):
                new_data[f"{team}{i}"].append(line[i-1])
        return pd.DataFrame(new_data, index=ser.index)
    data = pd.concat(
        [
            data,
            prep_heroes(data_json.radiant_heroes , "radiant" ),
            prep_heroes(data_json.dire_heroes , "dire" )
        ], axis=1
    )
    data_json.drop(["radiant_heroes", "dire_heroes"], axis=1, inplace=True)
    
    def prep_final_items(ser):
        new_data = {f"fin_item_{i}":list() for i in range(1,10)}
        for line in ser:
            line = json.loads(line)
            for i in range(1,10):
                new_data[f"fin_item_{i}"].append(line[i-1])
        return pd.DataFrame(new_data, index=ser.index)
    
    data = pd.concat(
        [
            data,
            prep_final_items(data_json.final_items),
        ], axis=1
    )
    data_json.drop("final_items", axis=1, inplace=True)

    return data

def pipe_timeser(type_ = "train"):
    train_timeser = pd.read_csv( os.path.join(input_dir, "processed", f"{type_}_timeseries.csv"), index_col=0 )
    train_timeser.drop(
        [col for col in train_timeser.columns if col.endswith("mode")],
        axis=1,
        inplace=True
    )
    return train_timeser
    
def pipe_level(type_ = "train"):
    
    train_level = pd.read_csv( os.path.join(input_dir, "processed", f"{type_}_levelup.csv"), index_col=0 )
    
    return train_level

def load_data(up_func):
    return {
        "train" : reduce_mem_usage( up_func("train") ),
        "test" : reduce_mem_usage( up_func("test") )
    }

In [6]:
data_def = load_data(pipe)
data_time = load_data(pipe_timeser)
data_level = load_data(pipe_level)

Memory usage of dataframe is 57.91 MB
Memory usage after optimization is: 13.14 MB
Decreased by 77.3%
Memory usage of dataframe is 25.09 MB
Memory usage after optimization is: 5.61 MB
Decreased by 77.6%
Memory usage of dataframe is 64.77 MB
Memory usage after optimization is: 22.29 MB
Decreased by 65.6%
Memory usage of dataframe is 28.06 MB
Memory usage after optimization is: 9.49 MB
Decreased by 66.2%
Memory usage of dataframe is 11.43 MB
Memory usage after optimization is: 4.00 MB
Decreased by 65.0%
Memory usage of dataframe is 4.95 MB
Memory usage after optimization is: 1.73 MB
Decreased by 65.0%


In [7]:
base_features = ['assists', 'avg_assists_x16', 'avg_deaths_x16', 'avg_gpm_x16',
       'avg_kills_x16', 'avg_xpm_x16', 'best_assists_x16', 'best_gpm_x16',
       'best_kills_x16', 'best_win_streak', 'best_xpm_x16',
       'camps_stacked', 'deaths', 'denies', 'dire_barracks_status',
       'dire_tower_status', 'duration', 'farm_score', 'fight_score',
       'first_blood_time', 'gold', 'gold_per_min', 'gold_spent',
       'hero_id', 'hero_pick_order', 'kills', 'last_hits',
       'leaver_status', 'nearby_creep_death_count', 'net_worth',
       'observer_wards_placed', 'party_players', 'player_team',
       'pre_game_duration', 'push_score', 'radiant_barracks_status',
       'radiant_tower_status', 'roshan_kills', 'rune_pickups',
       'scaled_hero_damage', 'sentry_wards_placed', 'stuns',
       'support_score', 'team_fight_participation', 'tower_damage',
       'win_streak', 'xp_per_min', 'radiant1', 'radiant2', 'radiant3',
       'radiant4', 'radiant5', 'dire1', 'dire2', 'dire3', 'dire4',
       'dire5', 'fin_item_1', 'fin_item_2', 'fin_item_3', 'fin_item_4',
       'fin_item_5', 'fin_item_6', 'fin_item_7', 'fin_item_8',
       'fin_item_9']

In [8]:
score = cross_val_score(
    cb.CatBoostClassifier(logging_level="Silent"),
    pd.concat([
        data_def["train"],
        data_time["train"],
    ],axis=1),
    y_train,
    scoring="accuracy",
    cv = 4,
    verbose=0,
    n_jobs=-1
)
print(score, np.mean(score), np.std(score))

NameError: name 'cross_val_score' is not defined

In [ ]:
score = cross_val_score(
    cb.CatBoostClassifier(logging_level="Silent"),
    pd.concat([
        data_def["train"][base_features],
        data_time["train"],
    ],axis=1),
    y_train,
    scoring="accuracy",
    cv = 4,
    verbose=0,
    n_jobs=-1
)
print(score, np.mean(score), np.std(score))

In [ ]:
score = cross_val_score(
    cb.CatBoostClassifier(logging_level="Silent"),
    pd.concat([
        data_def["train"][base_features],
        data_time["train"],
        data_level["train"],
    ],axis=1),
    y_train,
    scoring="accuracy",
    cv = 4,
    verbose=0,
    n_jobs=-1
)
print(score, np.mean(score), np.std(score))

### Суммы по тому - какие возможности абилки качал

In [8]:
from sklearn.model_selection import cross_val_score, cross_val_predict
import catboost as cb
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression

In [221]:
model = cb.CatBoostClassifier(logging_level="Silent").fit(
    pd.concat([
        data_def["train"],
        data_time["train"],
    ],axis=1), 
    y_train)

y_pred = model.predict(
    pd.concat([
        data_def["test"],
        data_time["test"],
    ],axis=1)
)

y_pred = pd.DataFrame({'id' : data_def["test"].index, 'skilled': y_pred})
submit(y_pred, "cb_conveer_raw")

Learning rate set to 0.061655
File saved in : /home/furfa/work/ai-academy2019/predictions/cb_conveer_raw[2019-04-01 17:42:34.323087].csv


## Оставить в матчах только уникальные
unique_index = dict()
for ind in tqdm_notebook(data_match.index):
    selection = data_match
    
    for col in data_match.columns:
        selection = selection[ selection[col] == data_match.ix[ind, col] ]
    
    ind_list = selection.index
    unique_index[ind] = ind_list[0]

In [9]:
var_types = {
    name : p[0] for p in ( 
                            (ft.variable_types.Categorical, 
                             [
                                "dire_barracks_status", "dire_tower_status",
                                "radiant_tower_status", "radiant_barracks_status",
                                "radiant1","radiant2","radiant3","radiant4","radiant5",
                                "dire1","dire2","dire3","dire4", "dire5", "party_players", "roshan_kills", "hero_id",
                                 "fin_item_1","fin_item_2","fin_item_3","fin_item_4","fin_item_5",
                                 "fin_item_6","fin_item_7","fin_item_8","fin_item_9"
                             ]),
                            (ft.variable_types.Boolean, 
                             [
                                "winner_team", "first_blood_claimed",
                                 "leaver_status", "player_team"
                             ]),
                         ) for name in p[1]
}

In [10]:
def generate_features(data, var_types, 
                      trans_primitives=["multiply",'divide', "diff"], N_FEATURES=1000, 
                      shuffle=False):
    data = data.copy()
    
    start_columns = data.columns
    
    if shuffle:
        cols = data.columns.values
        np.random.shuffle(cols)
        data = data[cols]
    
    data = data.reset_index()
    data.id = data.id.astype(np.int32)
    
    N_FEATURES += data.shape[1]
    
    es = ft.EntitySet(id='players')
    
    main_entity_id = 'train_players'

    # Entities with a unique index
    es = es.entity_from_dataframe(
        entity_id=main_entity_id, 
        dataframe=data, # dataframe object
        index='id', # unique index
        variable_types=var_types
    )

    print(es)
    
    # DFS with specified primitives
    features, feature_names = ft.dfs(
        entityset=es, 
        target_entity=main_entity_id,
        trans_primitives = trans_primitives,
        agg_primitives=[], 
        max_depth=1, 
        features_only=False,
        verbose=True,
        chunk_size=0.5,
        max_features=N_FEATURES, # comment it later, computational burden reduction
         n_jobs=-1,
    )
    return features.drop(start_columns, axis=1)

In [11]:
def get_most_importance(df, y_train):
    model = cb.CatBoostClassifier(logging_level="Silent").fit(df, y_train)
    print(len(df.columns))
    print(len(model.feature_importances_))
    imp = pd.DataFrame({
        "name": df.columns,
        "importance" : model.feature_importances_,
    })
    imp = imp.sort_values(by="importance", ascending=False)
    
    
    score = cross_val_score(
            model,
            df[
                imp[:150].name.values
            ],
            y_train,
            scoring="accuracy",
            cv = 4,
            verbose=0,
            n_jobs=-1
    )
    print("mean: ",np.mean(score), "std: ", np.std(score))
    print(score)
    
    return imp

In [12]:
def generate_train_test(data, params):
    
    return {"train" : generate_features(
                            data["train"],
                            **params
                        ),
            "test" : generate_features(
                            data["test"],
                            **params
                        ),
           }

In [71]:
def get_important_df(df, all_columns):
    df_cols = set(df.columns)
    intersect = df_cols.intersection( set(all_columns) )
    return df[intersect]
def save_df(gen_func, base_name, path):
    gen_func("train").to_csv(
        os.path.join(path, f"train_{base_name}.csv")
    )
    print( os.path.join(path, f"train_{base_name}.csv") )
    gen_func("test").to_csv(
        os.path.join(path, f"test_{base_name}.csv")
    )
    print( os.path.join(path, f"test_{base_name}.csv") )

In [37]:
multi_features = generate_train_test(
    data_def,
    {
        "var_types" : var_types,
        "trans_primitives" : ["multiply"],
        "N_FEATURES" : 1000,
    }
)
imp1 = get_most_importance(
    pd.concat([
        multi_features["train"],
        data_def["train"],
    ], axis=1),
    y_train
)

Entityset: players
  Entities:
    train_players [Rows: 99871, Columns: 76]
  Relationships:
    No relationships
Built 1065 features
EntitySet scattered to workers in 0.690 seconds
Elapsed: 00:17 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 3/3 chunks


Future exception was never retrieved
future: <Future finished exception=CommClosedError('in <closed TCP>: Stream is closed',)>
Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 179, in read
    n_frames = yield stream.read_bytes(8)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1107, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 200, in read
    convert_stream_closed_error(self, e)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 128, in convert_stream_closed_error
    raise Comm

Entityset: players
  Entities:
    train_players [Rows: 43265, Columns: 76]
  Relationships:
    No relationships
Built 1065 features
EntitySet scattered to workers in 0.537 seconds
Elapsed: 00:08 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 3/3 chunks


Exception ignored in: <generator object add_client at 0x7f0126fe6f68>
RuntimeError: generator ignored GeneratorExit


Learning rate set to 0.061655


Exception ignored in: <generator object add_client at 0x7f0125fbf8e0>
RuntimeError: generator ignored GeneratorExit
Future exception was never retrieved
future: <Future finished exception=CommClosedError('in <closed TCP>: Stream is closed',)>
Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 179, in read
    n_frames = yield stream.read_bytes(8)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1107, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 200, in read
    convert_stream_closed_error(self, e)
  File "/home/furfa/ana

1065
1065
Learning rate set to 0.056834
Learning rate set to 0.056834
Learning rate set to 0.056834
Learning rate set to 0.056834
mean:  0.7725566086708983 std:  0.0016952945641478872
[0.77126722 0.77058635 0.77475168 0.77362118]


In [47]:
most_important_multi = {t : get_important_df(multi_features[ t ], imp1[:150].name.values)
                        for t in ("train", "test")
                       }

In [15]:
divide_features = generate_train_test(
    data_def,
    {
        "var_types" : var_types,
        "trans_primitives" : ["divide"],
        "N_FEATURES" : 1000,
    }
)
imp2 = get_most_importance(
    pd.concat([
        divide_features["train"],
        data_def["train"],
    ], axis=1),
    y_train
)

Entityset: players
  Entities:
    train_players [Rows: 99871, Columns: 76]
  Relationships:
    No relationships
Built 1076 features
EntitySet scattered to workers in 0.646 seconds
Elapsed: 00:13 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 3/3 chunks


Future exception was never retrieved
future: <Future finished exception=StreamClosedError('Stream is closed',)>
Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1107, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/tcpclient.py", line 232, in connect
    af, addr, stream = yield connector.start(connect_timeout=timeout)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/tcpclient.py", line 112, in on_connect_done
    stream = future.result()
tornado.iostream.StreamClosedError: Stream is closed


Entityset: players
  Entities:
    train_players [Rows: 43265, Columns: 76]
  Relationships:
    No relationships
Built 1076 features
EntitySet scattered to workers in 0.556 seconds
Elapsed: 00:06 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 3/3 chunks
Learning rate set to 0.061655
1076
1076
Learning rate set to 0.056834
Learning rate set to 0.056834
Learning rate set to 0.056834
Learning rate set to 0.056834
mean:  0.7785944019319215 std:  0.0015451335770335487
[0.7768744  0.77723486 0.7801586  0.78010974]


In [48]:
most_important_divide = {t : get_important_df(divide_features[ t ], imp2[:150].name.values)
                        for t in ("train", "test")
                       }

In [16]:
divide_features_time = generate_train_test(
    data_time,
    {
        "var_types" : {},
        "trans_primitives" : ["divide"],
        "N_FEATURES" : 1000,
    }
)

imp3 = get_most_importance(
    pd.concat([
        divide_features_time["train"],
        data_def["train"],
    ], axis=1),
    y_train
)

Entityset: players
  Entities:
    train_players [Rows: 99871, Columns: 85]
  Relationships:
    No relationships
Built 1085 features
EntitySet scattered to workers in 0.678 seconds
Elapsed: 00:13 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 3/3 chunks


Future exception was never retrieved
future: <Future finished exception=CommClosedError('in <closed TCP>: Stream is closed',)>
Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 179, in read
    n_frames = yield stream.read_bytes(8)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1107, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 200, in read
    convert_stream_closed_error(self, e)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 128, in convert_stream_closed_error
    raise Comm

Entityset: players
  Entities:
    train_players [Rows: 43265, Columns: 85]
  Relationships:
    No relationships
Built 1085 features
EntitySet scattered to workers in 0.566 seconds
Elapsed: 00:06 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 3/3 chunks
Learning rate set to 0.061655


Future exception was never retrieved
future: <Future finished exception=StreamClosedError('Stream is closed',)>
Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1107, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/tcpclient.py", line 232, in connect
    af, addr, stream = yield connector.start(connect_timeout=timeout)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/tcpclient.py", line 112, in on_connect_done
    stream = future.result()
tornado.iostream.StreamClosedError: Stream is closed


1076
1076
Learning rate set to 0.056834
Learning rate set to 0.056834
Learning rate set to 0.056834
Learning rate set to 0.056834
mean:  0.8099548469022364 std:  0.0014703641608878777
[0.81087792 0.81103813 0.80743351 0.81046982]


In [50]:
most_important_divide_time = {t : get_important_df(divide_features_time[ t ], imp3[:150].name.values)
                        for t in ("train", "test")
                       }

In [17]:
multi_features_time = generate_train_test(
    data_time,
    {
        "var_types" : {},
        "trans_primitives" : ["multiply"],
        "N_FEATURES" : 1000,
    }
)

imp4 = get_most_importance(
    pd.concat([
        multi_features_time["train"],
        data_def["train"],
    ], axis=1),
    y_train
)

Entityset: players
  Entities:
    train_players [Rows: 99871, Columns: 85]
  Relationships:
    No relationships
Built 1085 features
EntitySet scattered to workers in 0.675 seconds
Elapsed: 00:15 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 3/3 chunks


Future exception was never retrieved
future: <Future finished exception=CommClosedError('in <closed TCP>: Stream is closed',)>
Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 179, in read
    n_frames = yield stream.read_bytes(8)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1107, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 200, in read
    convert_stream_closed_error(self, e)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 128, in convert_stream_closed_error
    raise Comm

Entityset: players
  Entities:
    train_players [Rows: 43265, Columns: 85]
  Relationships:
    No relationships
Built 1085 features


Exception ignored in: <generator object add_client at 0x7f01280f8eb8>
RuntimeError: generator ignored GeneratorExit
Future exception was never retrieved
future: <Future finished exception=CommClosedError('in <closed TCP>: Stream is closed',)>
Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 179, in read
    n_frames = yield stream.read_bytes(8)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1107, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 200, in read
    convert_stream_closed_error(self, e)
  File "/home/furfa/ana

EntitySet scattered to workers in 0.634 seconds
Elapsed: 00:07 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 3/3 chunks
Learning rate set to 0.061655


Exception ignored in: <generator object add_client at 0x7f0127ee9570>
RuntimeError: generator ignored GeneratorExit
Future exception was never retrieved
future: <Future finished exception=CommClosedError('in <closed TCP>: Stream is closed',)>
Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 179, in read
    n_frames = yield stream.read_bytes(8)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/tornado/gen.py", line 1107, in run
    yielded = self.gen.throw(*exc_info)
  File "/home/furfa/anaconda3/lib/python3.6/site-packages/distributed/comm/tcp.py", line 200, in read
    convert_stream_closed_error(self, e)
  File "/home/furfa/ana

1076
1076
Learning rate set to 0.056834
Learning rate set to 0.056834
Learning rate set to 0.056834
Learning rate set to 0.056834
mean:  0.8059096053089342 std:  0.0019155289434455343
[0.80731336 0.80827459 0.80394905 0.80410141]


In [52]:
most_important_multi_time = {t : get_important_df(multi_features_time[ t ], imp4[:150].name.values)
                        for t in ("train", "test")
                       }

In [30]:
model = cb.CatBoostClassifier(iterations=10000).fit(
    pd.concat([
        data_def["train"],
        
        divide_features_time["train"],
    ],axis=1)[top_columns], 
    y_train)

y_pred = model.predict(
    pd.concat([
        data_def["test"],
        divide_features_time["test"],
    ],axis=1)[top_columns]
)

y_pred = pd.DataFrame({'id' : data_def["test"].index, 'skilled': y_pred})
submit(y_pred, "time_divide_cb_best")

Learning rate set to 0.007924
0:	learn: 0.6911346	total: 36.6ms	remaining: 6m 5s
1:	learn: 0.6891182	total: 72.9ms	remaining: 6m 4s
2:	learn: 0.6871602	total: 108ms	remaining: 5m 59s
3:	learn: 0.6852021	total: 143ms	remaining: 5m 56s
4:	learn: 0.6832965	total: 178ms	remaining: 5m 54s
5:	learn: 0.6814260	total: 212ms	remaining: 5m 53s
6:	learn: 0.6795815	total: 247ms	remaining: 5m 53s
7:	learn: 0.6777959	total: 283ms	remaining: 5m 53s
8:	learn: 0.6760026	total: 318ms	remaining: 5m 52s
9:	learn: 0.6742437	total: 353ms	remaining: 5m 52s
10:	learn: 0.6725356	total: 387ms	remaining: 5m 51s
11:	learn: 0.6708843	total: 422ms	remaining: 5m 51s
12:	learn: 0.6692163	total: 457ms	remaining: 5m 51s
13:	learn: 0.6675635	total: 493ms	remaining: 5m 51s
14:	learn: 0.6659465	total: 528ms	remaining: 5m 51s
15:	learn: 0.6643652	total: 563ms	remaining: 5m 51s
16:	learn: 0.6628071	total: 598ms	remaining: 5m 51s
17:	learn: 0.6612483	total: 633ms	remaining: 5m 51s
18:	learn: 0.6597378	total: 669ms	remaining:

162:	learn: 0.5594926	total: 5.79s	remaining: 5m 49s
163:	learn: 0.5591772	total: 5.83s	remaining: 5m 49s
164:	learn: 0.5588754	total: 5.86s	remaining: 5m 49s
165:	learn: 0.5585823	total: 5.89s	remaining: 5m 49s
166:	learn: 0.5582763	total: 5.93s	remaining: 5m 49s
167:	learn: 0.5579389	total: 5.96s	remaining: 5m 49s
168:	learn: 0.5576225	total: 6s	remaining: 5m 49s
169:	learn: 0.5571968	total: 6.04s	remaining: 5m 49s
170:	learn: 0.5568708	total: 6.07s	remaining: 5m 49s
171:	learn: 0.5565954	total: 6.11s	remaining: 5m 48s
172:	learn: 0.5562757	total: 6.14s	remaining: 5m 48s
173:	learn: 0.5559867	total: 6.18s	remaining: 5m 48s
174:	learn: 0.5556842	total: 6.21s	remaining: 5m 48s
175:	learn: 0.5553015	total: 6.25s	remaining: 5m 48s
176:	learn: 0.5549605	total: 6.28s	remaining: 5m 48s
177:	learn: 0.5546770	total: 6.32s	remaining: 5m 48s
178:	learn: 0.5543971	total: 6.35s	remaining: 5m 48s
179:	learn: 0.5540987	total: 6.39s	remaining: 5m 48s
180:	learn: 0.5538349	total: 6.42s	remaining: 5m 

318:	learn: 0.5216505	total: 11.3s	remaining: 5m 44s
319:	learn: 0.5215066	total: 11.4s	remaining: 5m 44s
320:	learn: 0.5213204	total: 11.4s	remaining: 5m 43s
321:	learn: 0.5211743	total: 11.4s	remaining: 5m 43s
322:	learn: 0.5210330	total: 11.5s	remaining: 5m 43s
323:	learn: 0.5208803	total: 11.5s	remaining: 5m 43s
324:	learn: 0.5207433	total: 11.5s	remaining: 5m 43s
325:	learn: 0.5206095	total: 11.6s	remaining: 5m 43s
326:	learn: 0.5203960	total: 11.6s	remaining: 5m 43s
327:	learn: 0.5202643	total: 11.7s	remaining: 5m 43s
328:	learn: 0.5200527	total: 11.7s	remaining: 5m 43s
329:	learn: 0.5199103	total: 11.7s	remaining: 5m 43s
330:	learn: 0.5197566	total: 11.8s	remaining: 5m 43s
331:	learn: 0.5195903	total: 11.8s	remaining: 5m 43s
332:	learn: 0.5194464	total: 11.8s	remaining: 5m 43s
333:	learn: 0.5192780	total: 11.9s	remaining: 5m 43s
334:	learn: 0.5191151	total: 11.9s	remaining: 5m 43s
335:	learn: 0.5189761	total: 11.9s	remaining: 5m 43s
336:	learn: 0.5188290	total: 12s	remaining: 5m

474:	learn: 0.5003811	total: 16.9s	remaining: 5m 38s
475:	learn: 0.5002891	total: 16.9s	remaining: 5m 38s
476:	learn: 0.5001679	total: 17s	remaining: 5m 38s
477:	learn: 0.5000516	total: 17s	remaining: 5m 38s
478:	learn: 0.4999538	total: 17s	remaining: 5m 38s
479:	learn: 0.4998603	total: 17.1s	remaining: 5m 38s
480:	learn: 0.4997767	total: 17.1s	remaining: 5m 38s
481:	learn: 0.4996649	total: 17.1s	remaining: 5m 38s
482:	learn: 0.4995553	total: 17.2s	remaining: 5m 38s
483:	learn: 0.4994645	total: 17.2s	remaining: 5m 38s
484:	learn: 0.4993365	total: 17.3s	remaining: 5m 38s
485:	learn: 0.4992202	total: 17.3s	remaining: 5m 38s
486:	learn: 0.4990943	total: 17.3s	remaining: 5m 38s
487:	learn: 0.4990007	total: 17.4s	remaining: 5m 38s
488:	learn: 0.4989132	total: 17.4s	remaining: 5m 38s
489:	learn: 0.4987946	total: 17.4s	remaining: 5m 38s
490:	learn: 0.4987047	total: 17.5s	remaining: 5m 38s
491:	learn: 0.4986074	total: 17.5s	remaining: 5m 38s
492:	learn: 0.4985214	total: 17.5s	remaining: 5m 38s

630:	learn: 0.4858688	total: 22.5s	remaining: 5m 33s
631:	learn: 0.4857808	total: 22.5s	remaining: 5m 33s
632:	learn: 0.4857032	total: 22.5s	remaining: 5m 33s
633:	learn: 0.4856217	total: 22.6s	remaining: 5m 33s
634:	learn: 0.4855499	total: 22.6s	remaining: 5m 33s
635:	learn: 0.4854595	total: 22.7s	remaining: 5m 33s
636:	learn: 0.4853782	total: 22.7s	remaining: 5m 33s
637:	learn: 0.4852861	total: 22.7s	remaining: 5m 33s
638:	learn: 0.4852199	total: 22.8s	remaining: 5m 33s
639:	learn: 0.4851646	total: 22.8s	remaining: 5m 33s
640:	learn: 0.4850991	total: 22.8s	remaining: 5m 33s
641:	learn: 0.4850178	total: 22.9s	remaining: 5m 33s
642:	learn: 0.4849470	total: 22.9s	remaining: 5m 33s
643:	learn: 0.4848878	total: 22.9s	remaining: 5m 33s
644:	learn: 0.4848143	total: 23s	remaining: 5m 33s
645:	learn: 0.4847054	total: 23s	remaining: 5m 33s
646:	learn: 0.4846250	total: 23s	remaining: 5m 33s
647:	learn: 0.4845556	total: 23.1s	remaining: 5m 33s
648:	learn: 0.4844921	total: 23.1s	remaining: 5m 33s

...

9368:	learn: 0.3523263	total: 5m 36s	remaining: 22.7s
9369:	learn: 0.3523190	total: 5m 36s	remaining: 22.6s
9370:	learn: 0.3523106	total: 5m 36s	remaining: 22.6s
9371:	learn: 0.3523056	total: 5m 36s	remaining: 22.6s
9372:	learn: 0.3522961	total: 5m 36s	remaining: 22.5s
9373:	learn: 0.3522903	total: 5m 36s	remaining: 22.5s
9374:	learn: 0.3522805	total: 5m 36s	remaining: 22.5s
9375:	learn: 0.3522700	total: 5m 36s	remaining: 22.4s
9376:	learn: 0.3522589	total: 5m 36s	remaining: 22.4s
9377:	learn: 0.3522525	total: 5m 36s	remaining: 22.3s
9378:	learn: 0.3522413	total: 5m 36s	remaining: 22.3s
9379:	learn: 0.3522337	total: 5m 37s	remaining: 22.3s
9380:	learn: 0.3522262	total: 5m 37s	remaining: 22.2s
9381:	learn: 0.3522181	total: 5m 37s	remaining: 22.2s
9382:	learn: 0.3522105	total: 5m 37s	remaining: 22.2s
9383:	learn: 0.3522049	total: 5m 37s	remaining: 22.1s
9384:	learn: 0.3521929	total: 5m 37s	remaining: 22.1s
9385:	learn: 0.3521837	total: 5m 37s	remaining: 22.1s
9386:	learn: 0.3521784	total

9526:	learn: 0.3511284	total: 5m 42s	remaining: 17s
9527:	learn: 0.3511200	total: 5m 42s	remaining: 17s
9528:	learn: 0.3511124	total: 5m 42s	remaining: 16.9s
9529:	learn: 0.3511050	total: 5m 42s	remaining: 16.9s
9530:	learn: 0.3510979	total: 5m 42s	remaining: 16.9s
9531:	learn: 0.3510912	total: 5m 42s	remaining: 16.8s
9532:	learn: 0.3510870	total: 5m 42s	remaining: 16.8s
9533:	learn: 0.3510797	total: 5m 42s	remaining: 16.7s
9534:	learn: 0.3510749	total: 5m 42s	remaining: 16.7s
9535:	learn: 0.3510654	total: 5m 42s	remaining: 16.7s
9536:	learn: 0.3510588	total: 5m 42s	remaining: 16.6s
9537:	learn: 0.3510517	total: 5m 42s	remaining: 16.6s
9538:	learn: 0.3510459	total: 5m 42s	remaining: 16.6s
9539:	learn: 0.3510387	total: 5m 42s	remaining: 16.5s
9540:	learn: 0.3510335	total: 5m 42s	remaining: 16.5s
9541:	learn: 0.3510244	total: 5m 42s	remaining: 16.5s
9542:	learn: 0.3510158	total: 5m 42s	remaining: 16.4s
9543:	learn: 0.3510073	total: 5m 42s	remaining: 16.4s
9544:	learn: 0.3510007	total: 5m

9679:	learn: 0.3500038	total: 5m 47s	remaining: 11.5s
9680:	learn: 0.3499975	total: 5m 47s	remaining: 11.5s
9681:	learn: 0.3499901	total: 5m 47s	remaining: 11.4s
9682:	learn: 0.3499746	total: 5m 47s	remaining: 11.4s
9683:	learn: 0.3499681	total: 5m 47s	remaining: 11.4s
9684:	learn: 0.3499614	total: 5m 48s	remaining: 11.3s
9685:	learn: 0.3499549	total: 5m 48s	remaining: 11.3s
9686:	learn: 0.3499478	total: 5m 48s	remaining: 11.2s
9687:	learn: 0.3499411	total: 5m 48s	remaining: 11.2s
9688:	learn: 0.3499347	total: 5m 48s	remaining: 11.2s
9689:	learn: 0.3499247	total: 5m 48s	remaining: 11.1s
9690:	learn: 0.3499184	total: 5m 48s	remaining: 11.1s
9691:	learn: 0.3499110	total: 5m 48s	remaining: 11.1s
9692:	learn: 0.3499035	total: 5m 48s	remaining: 11s
9693:	learn: 0.3498956	total: 5m 48s	remaining: 11s
9694:	learn: 0.3498888	total: 5m 48s	remaining: 11s
9695:	learn: 0.3498812	total: 5m 48s	remaining: 10.9s
9696:	learn: 0.3498730	total: 5m 48s	remaining: 10.9s
9697:	learn: 0.3498654	total: 5m 4

9837:	learn: 0.3488366	total: 5m 53s	remaining: 5.82s
9838:	learn: 0.3488287	total: 5m 53s	remaining: 5.79s
9839:	learn: 0.3488249	total: 5m 53s	remaining: 5.75s
9840:	learn: 0.3488207	total: 5m 53s	remaining: 5.71s
9841:	learn: 0.3488102	total: 5m 53s	remaining: 5.68s
9842:	learn: 0.3488033	total: 5m 53s	remaining: 5.64s
9843:	learn: 0.3487950	total: 5m 53s	remaining: 5.61s
9844:	learn: 0.3487835	total: 5m 53s	remaining: 5.57s
9845:	learn: 0.3487766	total: 5m 53s	remaining: 5.53s
9846:	learn: 0.3487712	total: 5m 53s	remaining: 5.5s
9847:	learn: 0.3487648	total: 5m 53s	remaining: 5.46s
9848:	learn: 0.3487579	total: 5m 53s	remaining: 5.43s
9849:	learn: 0.3487501	total: 5m 53s	remaining: 5.39s
9850:	learn: 0.3487417	total: 5m 53s	remaining: 5.35s
9851:	learn: 0.3487333	total: 5m 54s	remaining: 5.32s
9852:	learn: 0.3487261	total: 5m 54s	remaining: 5.28s
9853:	learn: 0.3487187	total: 5m 54s	remaining: 5.25s
9854:	learn: 0.3487122	total: 5m 54s	remaining: 5.21s
9855:	learn: 0.3487055	total:

9989:	learn: 0.3477033	total: 5m 59s	remaining: 359ms
9990:	learn: 0.3476964	total: 5m 59s	remaining: 323ms
9991:	learn: 0.3476883	total: 5m 59s	remaining: 287ms
9992:	learn: 0.3476779	total: 5m 59s	remaining: 252ms
9993:	learn: 0.3476716	total: 5m 59s	remaining: 216ms
9994:	learn: 0.3476641	total: 5m 59s	remaining: 180ms
9995:	learn: 0.3476579	total: 5m 59s	remaining: 144ms
9996:	learn: 0.3476501	total: 5m 59s	remaining: 108ms
9997:	learn: 0.3476420	total: 5m 59s	remaining: 71.9ms
9998:	learn: 0.3476326	total: 5m 59s	remaining: 35.9ms
9999:	learn: 0.3476269	total: 5m 59s	remaining: 0us
File saved in : /home/furfa/work/ai-academy2019/predictions/time_divide_cb_best[2019-04-02 22:40:55.287189].csv


In [61]:
t = "train"

df = lambda t : pd.concat([
                    data_def[ t ],
                    data_time[ t ],
                    most_important_divide[ t ],
                    most_important_multi[ t ],
                    most_important_divide_time[ t ],
                    most_important_multi_time[ t ],
                    data_level[ t ],
                ],axis=1)

score = cross_val_score(
    cb.CatBoostClassifier(logging_level="Silent"),
    df(t),
    y_train,
    scoring="accuracy",
    cv = 4,
    verbose=0,
    n_jobs=-1
)

print("mean: ",np.mean(score), "std: ", np.std(score))
print(score)

In [64]:
t = "train"
model = cb.CatBoostClassifier(iterations=20000).fit(
    df(t), 
    y_train)

t = "test"
y_pred = model.predict(
    df(t)
)

y_pred = pd.DataFrame({'id' : data_def["test"].index, 'skilled': y_pred})
submit(y_pred, "top_df_cb20000")

Learning rate set to 0.004273
0:	learn: 0.6919894	total: 73.1ms	remaining: 24m 22s
1:	learn: 0.6908150	total: 149ms	remaining: 24m 50s
2:	learn: 0.6896639	total: 222ms	remaining: 24m 39s
3:	learn: 0.6885432	total: 294ms	remaining: 24m 28s
4:	learn: 0.6874288	total: 373ms	remaining: 24m 50s
5:	learn: 0.6863330	total: 448ms	remaining: 24m 51s
6:	learn: 0.6852135	total: 519ms	remaining: 24m 42s
7:	learn: 0.6841344	total: 593ms	remaining: 24m 41s
8:	learn: 0.6830624	total: 664ms	remaining: 24m 34s
9:	learn: 0.6819566	total: 736ms	remaining: 24m 30s
10:	learn: 0.6809056	total: 808ms	remaining: 24m 27s
11:	learn: 0.6798632	total: 880ms	remaining: 24m 25s
12:	learn: 0.6788548	total: 952ms	remaining: 24m 23s
13:	learn: 0.6778345	total: 1.03s	remaining: 24m 25s
14:	learn: 0.6768434	total: 1.1s	remaining: 24m 23s
15:	learn: 0.6758376	total: 1.17s	remaining: 24m 21s
16:	learn: 0.6748001	total: 1.25s	remaining: 24m 23s
17:	learn: 0.6738212	total: 1.31s	remaining: 24m 20s
18:	learn: 0.6728031	total

156:	learn: 0.5896591	total: 11.4s	remaining: 23m 59s
157:	learn: 0.5892446	total: 11.5s	remaining: 23m 59s
158:	learn: 0.5888575	total: 11.5s	remaining: 23m 58s
159:	learn: 0.5884860	total: 11.6s	remaining: 23m 58s
160:	learn: 0.5880763	total: 11.7s	remaining: 23m 58s
161:	learn: 0.5877384	total: 11.7s	remaining: 23m 58s
162:	learn: 0.5873333	total: 11.8s	remaining: 23m 57s
163:	learn: 0.5869757	total: 11.9s	remaining: 23m 57s
164:	learn: 0.5866396	total: 12s	remaining: 23m 57s
165:	learn: 0.5862486	total: 12s	remaining: 23m 57s
166:	learn: 0.5859252	total: 12.1s	remaining: 23m 57s
167:	learn: 0.5855487	total: 12.2s	remaining: 23m 56s
168:	learn: 0.5851654	total: 12.2s	remaining: 23m 56s
169:	learn: 0.5848122	total: 12.3s	remaining: 23m 56s
170:	learn: 0.5844105	total: 12.4s	remaining: 23m 56s
171:	learn: 0.5840863	total: 12.5s	remaining: 23m 56s
172:	learn: 0.5837650	total: 12.5s	remaining: 23m 56s
173:	learn: 0.5834343	total: 12.6s	remaining: 23m 56s
174:	learn: 0.5830949	total: 12.

309:	learn: 0.5476431	total: 22.4s	remaining: 23m 44s
310:	learn: 0.5474127	total: 22.5s	remaining: 23m 44s
311:	learn: 0.5472521	total: 22.6s	remaining: 23m 44s
312:	learn: 0.5470756	total: 22.6s	remaining: 23m 44s
313:	learn: 0.5468943	total: 22.7s	remaining: 23m 44s
314:	learn: 0.5467077	total: 22.8s	remaining: 23m 43s
315:	learn: 0.5465266	total: 22.9s	remaining: 23m 43s
316:	learn: 0.5463000	total: 22.9s	remaining: 23m 43s
317:	learn: 0.5461219	total: 23s	remaining: 23m 43s
318:	learn: 0.5459282	total: 23.1s	remaining: 23m 43s
319:	learn: 0.5457451	total: 23.1s	remaining: 23m 43s
320:	learn: 0.5455111	total: 23.2s	remaining: 23m 43s
321:	learn: 0.5453145	total: 23.3s	remaining: 23m 43s
322:	learn: 0.5451500	total: 23.4s	remaining: 23m 43s
323:	learn: 0.5449805	total: 23.4s	remaining: 23m 43s
324:	learn: 0.5448100	total: 23.5s	remaining: 23m 43s
325:	learn: 0.5446359	total: 23.6s	remaining: 23m 43s
326:	learn: 0.5443935	total: 23.7s	remaining: 23m 43s
327:	learn: 0.5442278	total: 2

462:	learn: 0.5238515	total: 33.5s	remaining: 23m 35s
463:	learn: 0.5237330	total: 33.6s	remaining: 23m 34s
464:	learn: 0.5236129	total: 33.7s	remaining: 23m 34s
465:	learn: 0.5235094	total: 33.8s	remaining: 23m 34s
466:	learn: 0.5233476	total: 33.8s	remaining: 23m 34s
467:	learn: 0.5232344	total: 33.9s	remaining: 23m 34s
468:	learn: 0.5231040	total: 34s	remaining: 23m 34s
469:	learn: 0.5229982	total: 34s	remaining: 23m 34s
470:	learn: 0.5228757	total: 34.1s	remaining: 23m 34s
471:	learn: 0.5227668	total: 34.2s	remaining: 23m 34s
472:	learn: 0.5226158	total: 34.3s	remaining: 23m 34s
473:	learn: 0.5224912	total: 34.3s	remaining: 23m 34s
474:	learn: 0.5223785	total: 34.4s	remaining: 23m 34s
475:	learn: 0.5222214	total: 34.5s	remaining: 23m 34s
476:	learn: 0.5221247	total: 34.5s	remaining: 23m 34s
477:	learn: 0.5220139	total: 34.6s	remaining: 23m 34s
478:	learn: 0.5218826	total: 34.7s	remaining: 23m 33s
479:	learn: 0.5217418	total: 34.8s	remaining: 23m 33s
480:	learn: 0.5216282	total: 34.

615:	learn: 0.5076184	total: 44.6s	remaining: 23m 24s
616:	learn: 0.5075217	total: 44.7s	remaining: 23m 24s
617:	learn: 0.5074412	total: 44.8s	remaining: 23m 24s
618:	learn: 0.5073606	total: 44.9s	remaining: 23m 24s
619:	learn: 0.5072466	total: 44.9s	remaining: 23m 24s
620:	learn: 0.5071476	total: 45s	remaining: 23m 24s
621:	learn: 0.5070443	total: 45.1s	remaining: 23m 24s
622:	learn: 0.5069276	total: 45.1s	remaining: 23m 24s
623:	learn: 0.5068395	total: 45.2s	remaining: 23m 24s
624:	learn: 0.5067624	total: 45.3s	remaining: 23m 23s
625:	learn: 0.5066627	total: 45.4s	remaining: 23m 23s
626:	learn: 0.5065575	total: 45.4s	remaining: 23m 23s
627:	learn: 0.5064704	total: 45.5s	remaining: 23m 23s
628:	learn: 0.5063875	total: 45.6s	remaining: 23m 23s
629:	learn: 0.5063098	total: 45.6s	remaining: 23m 23s
630:	learn: 0.5062193	total: 45.7s	remaining: 23m 23s
631:	learn: 0.5061039	total: 45.8s	remaining: 23m 23s
632:	learn: 0.5060079	total: 45.9s	remaining: 23m 23s
633:	learn: 0.5059015	total: 4

...

19395:	learn: 0.3334572	total: 23m 48s	remaining: 44.5s
19396:	learn: 0.3334528	total: 23m 48s	remaining: 44.4s
19397:	learn: 0.3334483	total: 23m 48s	remaining: 44.3s
19398:	learn: 0.3334441	total: 23m 48s	remaining: 44.3s
19399:	learn: 0.3334414	total: 23m 49s	remaining: 44.2s
19400:	learn: 0.3334372	total: 23m 49s	remaining: 44.1s
19401:	learn: 0.3334332	total: 23m 49s	remaining: 44s
19402:	learn: 0.3334294	total: 23m 49s	remaining: 44s
19403:	learn: 0.3334250	total: 23m 49s	remaining: 43.9s
19404:	learn: 0.3334209	total: 23m 49s	remaining: 43.8s
19405:	learn: 0.3334168	total: 23m 49s	remaining: 43.8s
19406:	learn: 0.3334134	total: 23m 49s	remaining: 43.7s
19407:	learn: 0.3334088	total: 23m 49s	remaining: 43.6s
19408:	learn: 0.3334052	total: 23m 49s	remaining: 43.5s
19409:	learn: 0.3334022	total: 23m 49s	remaining: 43.5s
19410:	learn: 0.3333983	total: 23m 49s	remaining: 43.4s
19411:	learn: 0.3333948	total: 23m 49s	remaining: 43.3s
19412:	learn: 0.3333900	total: 23m 49s	remaining: 43

19542:	learn: 0.3328733	total: 23m 59s	remaining: 33.7s
19543:	learn: 0.3328690	total: 23m 59s	remaining: 33.6s
19544:	learn: 0.3328657	total: 23m 59s	remaining: 33.5s
19545:	learn: 0.3328623	total: 23m 59s	remaining: 33.4s
19546:	learn: 0.3328581	total: 23m 59s	remaining: 33.4s
19547:	learn: 0.3328544	total: 23m 59s	remaining: 33.3s
19548:	learn: 0.3328506	total: 24m	remaining: 33.2s
19549:	learn: 0.3328466	total: 24m	remaining: 33.1s
19550:	learn: 0.3328427	total: 24m	remaining: 33.1s
19551:	learn: 0.3328391	total: 24m	remaining: 33s
19552:	learn: 0.3328356	total: 24m	remaining: 32.9s
19553:	learn: 0.3328317	total: 24m	remaining: 32.9s
19554:	learn: 0.3328271	total: 24m	remaining: 32.8s
19555:	learn: 0.3328244	total: 24m	remaining: 32.7s
19556:	learn: 0.3328196	total: 24m	remaining: 32.6s
19557:	learn: 0.3328170	total: 24m	remaining: 32.6s
19558:	learn: 0.3328128	total: 24m	remaining: 32.5s
19559:	learn: 0.3328080	total: 24m	remaining: 32.4s
19560:	learn: 0.3328037	total: 24m	remaini

19692:	learn: 0.3323064	total: 24m 10s	remaining: 22.6s
19693:	learn: 0.3323026	total: 24m 10s	remaining: 22.5s
19694:	learn: 0.3322983	total: 24m 10s	remaining: 22.5s
19695:	learn: 0.3322938	total: 24m 10s	remaining: 22.4s
19696:	learn: 0.3322909	total: 24m 10s	remaining: 22.3s
19697:	learn: 0.3322859	total: 24m 11s	remaining: 22.2s
19698:	learn: 0.3322833	total: 24m 11s	remaining: 22.2s
19699:	learn: 0.3322797	total: 24m 11s	remaining: 22.1s
19700:	learn: 0.3322770	total: 24m 11s	remaining: 22s
19701:	learn: 0.3322732	total: 24m 11s	remaining: 22s
19702:	learn: 0.3322706	total: 24m 11s	remaining: 21.9s
19703:	learn: 0.3322656	total: 24m 11s	remaining: 21.8s
19704:	learn: 0.3322623	total: 24m 11s	remaining: 21.7s
19705:	learn: 0.3322590	total: 24m 11s	remaining: 21.7s
19706:	learn: 0.3322540	total: 24m 11s	remaining: 21.6s
19707:	learn: 0.3322504	total: 24m 11s	remaining: 21.5s
19708:	learn: 0.3322467	total: 24m 11s	remaining: 21.4s
19709:	learn: 0.3322426	total: 24m 11s	remaining: 21

19839:	learn: 0.3317264	total: 24m 21s	remaining: 11.8s
19840:	learn: 0.3317234	total: 24m 21s	remaining: 11.7s
19841:	learn: 0.3317182	total: 24m 21s	remaining: 11.6s
19842:	learn: 0.3317138	total: 24m 21s	remaining: 11.6s
19843:	learn: 0.3317076	total: 24m 21s	remaining: 11.5s
19844:	learn: 0.3317041	total: 24m 21s	remaining: 11.4s
19845:	learn: 0.3316992	total: 24m 22s	remaining: 11.3s
19846:	learn: 0.3316957	total: 24m 22s	remaining: 11.3s
19847:	learn: 0.3316909	total: 24m 22s	remaining: 11.2s
19848:	learn: 0.3316875	total: 24m 22s	remaining: 11.1s
19849:	learn: 0.3316835	total: 24m 22s	remaining: 11.1s
19850:	learn: 0.3316792	total: 24m 22s	remaining: 11s
19851:	learn: 0.3316755	total: 24m 22s	remaining: 10.9s
19852:	learn: 0.3316715	total: 24m 22s	remaining: 10.8s
19853:	learn: 0.3316686	total: 24m 22s	remaining: 10.8s
19854:	learn: 0.3316645	total: 24m 22s	remaining: 10.7s
19855:	learn: 0.3316606	total: 24m 22s	remaining: 10.6s
19856:	learn: 0.3316577	total: 24m 22s	remaining: 

19986:	learn: 0.3311419	total: 24m 32s	remaining: 958ms
19987:	learn: 0.3311362	total: 24m 32s	remaining: 884ms
19988:	learn: 0.3311327	total: 24m 32s	remaining: 810ms
19989:	learn: 0.3311266	total: 24m 32s	remaining: 737ms
19990:	learn: 0.3311234	total: 24m 32s	remaining: 663ms
19991:	learn: 0.3311200	total: 24m 32s	remaining: 589ms
19992:	learn: 0.3311152	total: 24m 32s	remaining: 516ms
19993:	learn: 0.3311117	total: 24m 33s	remaining: 442ms
19994:	learn: 0.3311081	total: 24m 33s	remaining: 368ms
19995:	learn: 0.3311049	total: 24m 33s	remaining: 295ms
19996:	learn: 0.3311006	total: 24m 33s	remaining: 221ms
19997:	learn: 0.3310951	total: 24m 33s	remaining: 147ms
19998:	learn: 0.3310915	total: 24m 33s	remaining: 73.7ms
19999:	learn: 0.3310877	total: 24m 33s	remaining: 0us
File saved in : /home/furfa/work/ai-academy2019/predictions/top_df_cb20000[2019-04-04 00:54:56.044170].csv


In [63]:
t = "train"
model = cb.CatBoostClassifier(iterations=10000).fit(
    df(t), 
    y_train)

t = "test"
y_pred = model.predict(
    df(t)
)

y_pred = pd.DataFrame({'id' : data_def["test"].index, 'skilled': y_pred})
submit(y_pred, "top_df_cb10000")

Learning rate set to 0.007924
0:	learn: 0.6910217	total: 74.7ms	remaining: 12m 27s
1:	learn: 0.6888727	total: 150ms	remaining: 12m 28s
2:	learn: 0.6867541	total: 223ms	remaining: 12m 24s
3:	learn: 0.6847290	total: 299ms	remaining: 12m 26s
4:	learn: 0.6827449	total: 373ms	remaining: 12m 26s
5:	learn: 0.6807068	total: 447ms	remaining: 12m 24s
6:	learn: 0.6787762	total: 522ms	remaining: 12m 24s
7:	learn: 0.6769662	total: 596ms	remaining: 12m 24s
8:	learn: 0.6751232	total: 671ms	remaining: 12m 24s
9:	learn: 0.6732981	total: 745ms	remaining: 12m 24s
10:	learn: 0.6715140	total: 819ms	remaining: 12m 24s
11:	learn: 0.6697601	total: 894ms	remaining: 12m 24s
12:	learn: 0.6679458	total: 967ms	remaining: 12m 23s
13:	learn: 0.6663238	total: 1.04s	remaining: 12m 22s
14:	learn: 0.6646572	total: 1.11s	remaining: 12m 22s
15:	learn: 0.6630173	total: 1.19s	remaining: 12m 22s
16:	learn: 0.6614322	total: 1.26s	remaining: 12m 22s
17:	learn: 0.6598454	total: 1.34s	remaining: 12m 23s
18:	learn: 0.6582528	tota

156:	learn: 0.5511261	total: 11.7s	remaining: 12m 16s
157:	learn: 0.5508021	total: 11.8s	remaining: 12m 16s
158:	learn: 0.5503290	total: 11.9s	remaining: 12m 15s
159:	learn: 0.5498733	total: 12s	remaining: 12m 15s
160:	learn: 0.5494735	total: 12s	remaining: 12m 15s
161:	learn: 0.5490512	total: 12.1s	remaining: 12m 15s
162:	learn: 0.5486371	total: 12.2s	remaining: 12m 15s
163:	learn: 0.5482861	total: 12.3s	remaining: 12m 15s
164:	learn: 0.5479745	total: 12.3s	remaining: 12m 15s
165:	learn: 0.5476393	total: 12.4s	remaining: 12m 15s
166:	learn: 0.5473384	total: 12.5s	remaining: 12m 15s
167:	learn: 0.5469160	total: 12.6s	remaining: 12m 15s
168:	learn: 0.5465194	total: 12.6s	remaining: 12m 15s
169:	learn: 0.5461208	total: 12.7s	remaining: 12m 15s
170:	learn: 0.5458127	total: 12.8s	remaining: 12m 15s
171:	learn: 0.5454435	total: 12.9s	remaining: 12m 14s
172:	learn: 0.5450840	total: 12.9s	remaining: 12m 14s
173:	learn: 0.5447301	total: 13s	remaining: 12m 14s
174:	learn: 0.5443592	total: 13.1s

312:	learn: 0.5107192	total: 23.4s	remaining: 12m 2s
313:	learn: 0.5104965	total: 23.4s	remaining: 12m 2s
314:	learn: 0.5102842	total: 23.5s	remaining: 12m 2s
315:	learn: 0.5101205	total: 23.6s	remaining: 12m 2s
316:	learn: 0.5099677	total: 23.7s	remaining: 12m 2s
317:	learn: 0.5098142	total: 23.7s	remaining: 12m 2s
318:	learn: 0.5096341	total: 23.8s	remaining: 12m 2s
319:	learn: 0.5094552	total: 23.9s	remaining: 12m 2s
320:	learn: 0.5092780	total: 24s	remaining: 12m 2s
321:	learn: 0.5091327	total: 24s	remaining: 12m 2s
322:	learn: 0.5089906	total: 24.1s	remaining: 12m 1s
323:	learn: 0.5087887	total: 24.2s	remaining: 12m 2s
324:	learn: 0.5086253	total: 24.3s	remaining: 12m 1s
325:	learn: 0.5084589	total: 24.3s	remaining: 12m 1s
326:	learn: 0.5082922	total: 24.4s	remaining: 12m 1s
327:	learn: 0.5080756	total: 24.5s	remaining: 12m 1s
328:	learn: 0.5079305	total: 24.5s	remaining: 12m 1s
329:	learn: 0.5077630	total: 24.6s	remaining: 12m 1s
330:	learn: 0.5075736	total: 24.7s	remaining: 12m 

468:	learn: 0.4889322	total: 35s	remaining: 11m 51s
469:	learn: 0.4888418	total: 35.1s	remaining: 11m 52s
470:	learn: 0.4887592	total: 35.2s	remaining: 11m 51s
471:	learn: 0.4886327	total: 35.3s	remaining: 11m 51s
472:	learn: 0.4885429	total: 35.3s	remaining: 11m 51s
473:	learn: 0.4884431	total: 35.4s	remaining: 11m 51s
474:	learn: 0.4883340	total: 35.5s	remaining: 11m 51s
475:	learn: 0.4882178	total: 35.6s	remaining: 11m 51s
476:	learn: 0.4881141	total: 35.6s	remaining: 11m 51s
477:	learn: 0.4880141	total: 35.7s	remaining: 11m 51s
478:	learn: 0.4879117	total: 35.8s	remaining: 11m 51s
479:	learn: 0.4877907	total: 35.9s	remaining: 11m 51s
480:	learn: 0.4876820	total: 35.9s	remaining: 11m 51s
481:	learn: 0.4875604	total: 36s	remaining: 11m 51s
482:	learn: 0.4874470	total: 36.1s	remaining: 11m 51s
483:	learn: 0.4873479	total: 36.2s	remaining: 11m 51s
484:	learn: 0.4872502	total: 36.2s	remaining: 11m 51s
485:	learn: 0.4871194	total: 36.3s	remaining: 11m 50s
486:	learn: 0.4869984	total: 36.

621:	learn: 0.4748572	total: 46.4s	remaining: 11m 39s
622:	learn: 0.4747491	total: 46.5s	remaining: 11m 39s
623:	learn: 0.4746806	total: 46.6s	remaining: 11m 39s
624:	learn: 0.4745948	total: 46.6s	remaining: 11m 39s
625:	learn: 0.4745108	total: 46.7s	remaining: 11m 39s
626:	learn: 0.4744082	total: 46.8s	remaining: 11m 39s
627:	learn: 0.4743376	total: 46.9s	remaining: 11m 39s
628:	learn: 0.4742686	total: 46.9s	remaining: 11m 39s
629:	learn: 0.4741968	total: 47s	remaining: 11m 39s
630:	learn: 0.4741082	total: 47.1s	remaining: 11m 39s
631:	learn: 0.4740343	total: 47.2s	remaining: 11m 39s
632:	learn: 0.4739537	total: 47.2s	remaining: 11m 39s
633:	learn: 0.4738722	total: 47.3s	remaining: 11m 39s
634:	learn: 0.4737994	total: 47.4s	remaining: 11m 38s
635:	learn: 0.4737228	total: 47.5s	remaining: 11m 38s
636:	learn: 0.4736250	total: 47.5s	remaining: 11m 38s
637:	learn: 0.4735562	total: 47.6s	remaining: 11m 38s
638:	learn: 0.4734780	total: 47.7s	remaining: 11m 38s
639:	learn: 0.4733909	total: 4

...

9366:	learn: 0.3415442	total: 11m 34s	remaining: 46.9s
9367:	learn: 0.3415362	total: 11m 34s	remaining: 46.8s
9368:	learn: 0.3415274	total: 11m 34s	remaining: 46.8s
9369:	learn: 0.3415214	total: 11m 34s	remaining: 46.7s
9370:	learn: 0.3415161	total: 11m 34s	remaining: 46.6s
9371:	learn: 0.3415118	total: 11m 34s	remaining: 46.5s
9372:	learn: 0.3415040	total: 11m 34s	remaining: 46.5s
9373:	learn: 0.3414961	total: 11m 34s	remaining: 46.4s
9374:	learn: 0.3414898	total: 11m 34s	remaining: 46.3s
9375:	learn: 0.3414795	total: 11m 34s	remaining: 46.2s
9376:	learn: 0.3414739	total: 11m 34s	remaining: 46.2s
9377:	learn: 0.3414654	total: 11m 35s	remaining: 46.1s
9378:	learn: 0.3414598	total: 11m 35s	remaining: 46s
9379:	learn: 0.3414535	total: 11m 35s	remaining: 45.9s
9380:	learn: 0.3414460	total: 11m 35s	remaining: 45.9s
9381:	learn: 0.3414393	total: 11m 35s	remaining: 45.8s
9382:	learn: 0.3414317	total: 11m 35s	remaining: 45.7s
9383:	learn: 0.3414241	total: 11m 35s	remaining: 45.7s
9384:	learn:

9516:	learn: 0.3404361	total: 11m 45s	remaining: 35.8s
9517:	learn: 0.3404300	total: 11m 45s	remaining: 35.7s
9518:	learn: 0.3404211	total: 11m 45s	remaining: 35.7s
9519:	learn: 0.3404135	total: 11m 45s	remaining: 35.6s
9520:	learn: 0.3404083	total: 11m 45s	remaining: 35.5s
9521:	learn: 0.3404019	total: 11m 45s	remaining: 35.4s
9522:	learn: 0.3403940	total: 11m 45s	remaining: 35.4s
9523:	learn: 0.3403884	total: 11m 45s	remaining: 35.3s
9524:	learn: 0.3403822	total: 11m 46s	remaining: 35.2s
9525:	learn: 0.3403739	total: 11m 46s	remaining: 35.1s
9526:	learn: 0.3403670	total: 11m 46s	remaining: 35.1s
9527:	learn: 0.3403587	total: 11m 46s	remaining: 35s
9528:	learn: 0.3403542	total: 11m 46s	remaining: 34.9s
9529:	learn: 0.3403476	total: 11m 46s	remaining: 34.8s
9530:	learn: 0.3403406	total: 11m 46s	remaining: 34.8s
9531:	learn: 0.3403328	total: 11m 46s	remaining: 34.7s
9532:	learn: 0.3403237	total: 11m 46s	remaining: 34.6s
9533:	learn: 0.3403156	total: 11m 46s	remaining: 34.5s
9534:	learn:

9666:	learn: 0.3393186	total: 11m 56s	remaining: 24.7s
9667:	learn: 0.3393089	total: 11m 56s	remaining: 24.6s
9668:	learn: 0.3393021	total: 11m 56s	remaining: 24.5s
9669:	learn: 0.3392953	total: 11m 56s	remaining: 24.5s
9670:	learn: 0.3392893	total: 11m 56s	remaining: 24.4s
9671:	learn: 0.3392823	total: 11m 56s	remaining: 24.3s
9672:	learn: 0.3392743	total: 11m 56s	remaining: 24.2s
9673:	learn: 0.3392687	total: 11m 57s	remaining: 24.2s
9674:	learn: 0.3392609	total: 11m 57s	remaining: 24.1s
9675:	learn: 0.3392552	total: 11m 57s	remaining: 24s
9676:	learn: 0.3392482	total: 11m 57s	remaining: 23.9s
9677:	learn: 0.3392320	total: 11m 57s	remaining: 23.9s
9678:	learn: 0.3392260	total: 11m 57s	remaining: 23.8s
9679:	learn: 0.3392189	total: 11m 57s	remaining: 23.7s
9680:	learn: 0.3392101	total: 11m 57s	remaining: 23.6s
9681:	learn: 0.3392035	total: 11m 57s	remaining: 23.6s
9682:	learn: 0.3391949	total: 11m 57s	remaining: 23.5s
9683:	learn: 0.3391854	total: 11m 57s	remaining: 23.4s
9684:	learn:

9819:	learn: 0.3381233	total: 12m 7s	remaining: 13.3s
9820:	learn: 0.3381179	total: 12m 7s	remaining: 13.3s
9821:	learn: 0.3381118	total: 12m 8s	remaining: 13.2s
9822:	learn: 0.3381040	total: 12m 8s	remaining: 13.1s
9823:	learn: 0.3380969	total: 12m 8s	remaining: 13s
9824:	learn: 0.3380868	total: 12m 8s	remaining: 13s
9825:	learn: 0.3380801	total: 12m 8s	remaining: 12.9s
9826:	learn: 0.3380721	total: 12m 8s	remaining: 12.8s
9827:	learn: 0.3380666	total: 12m 8s	remaining: 12.7s
9828:	learn: 0.3380577	total: 12m 8s	remaining: 12.7s
9829:	learn: 0.3380500	total: 12m 8s	remaining: 12.6s
9830:	learn: 0.3380413	total: 12m 8s	remaining: 12.5s
9831:	learn: 0.3380356	total: 12m 8s	remaining: 12.5s
9832:	learn: 0.3380276	total: 12m 8s	remaining: 12.4s
9833:	learn: 0.3380216	total: 12m 8s	remaining: 12.3s
9834:	learn: 0.3380134	total: 12m 9s	remaining: 12.2s
9835:	learn: 0.3380078	total: 12m 9s	remaining: 12.2s
9836:	learn: 0.3380031	total: 12m 9s	remaining: 12.1s
9837:	learn: 0.3379963	total: 12

9969:	learn: 0.3370382	total: 12m 19s	remaining: 2.22s
9970:	learn: 0.3370307	total: 12m 19s	remaining: 2.15s
9971:	learn: 0.3370227	total: 12m 19s	remaining: 2.08s
9972:	learn: 0.3370165	total: 12m 19s	remaining: 2s
9973:	learn: 0.3370061	total: 12m 19s	remaining: 1.93s
9974:	learn: 0.3370007	total: 12m 19s	remaining: 1.85s
9975:	learn: 0.3369938	total: 12m 19s	remaining: 1.78s
9976:	learn: 0.3369856	total: 12m 19s	remaining: 1.7s
9977:	learn: 0.3369775	total: 12m 19s	remaining: 1.63s
9978:	learn: 0.3369720	total: 12m 19s	remaining: 1.56s
9979:	learn: 0.3369634	total: 12m 19s	remaining: 1.48s
9980:	learn: 0.3369556	total: 12m 19s	remaining: 1.41s
9981:	learn: 0.3369460	total: 12m 19s	remaining: 1.33s
9982:	learn: 0.3369383	total: 12m 20s	remaining: 1.26s
9983:	learn: 0.3369258	total: 12m 20s	remaining: 1.19s
9984:	learn: 0.3369185	total: 12m 20s	remaining: 1.11s
9985:	learn: 0.3369093	total: 12m 20s	remaining: 1.04s
9986:	learn: 0.3369025	total: 12m 20s	remaining: 964ms
9987:	learn: 0